In [1]:
import torch, torchvision, pathlib
import numpy as np
import pyvips
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [22]:
trainImgPaths = pathlib.Path('/home/users/jeafilho/dataset/cvpr_chesapeake_landcover/').glob('*-train_tiles/*')
valImgPaths = pathlib.Path('/home/users/jeafilho/dataset/cvpr_chesapeake_landcover/').glob('*-val_tiles/*')
testImgPaths = pathlib.Path('/home/users/jeafilho/dataset/cvpr_chesapeake_landcover/').glob('*-test_tiles/*')

In [23]:
maskPaths = []
imgPaths = []
for p in sorted(trainImgPaths):
    if 'leaf-on' in p.stem:
        imgPaths.append(p)
    elif '_lc' in p.stem:
        maskPaths.append(p)

In [24]:
print(len(imgPaths), len(maskPaths))
for imgPath, maskPath in zip(imgPaths, maskPaths):
    img = pyvips.Image.new_from_file(str(imgPath))
    img = img.numpy()
    mask = pyvips.Image.new_from_file(str(maskPath))
    mask = mask.numpy()
    print(img.shape, mask.shape)
    break

582 582
/home/users/jeafilho/dataset/cvpr_chesapeake_landcover/de_1m_2013_extended-debuffered-train_tiles/m_3807503_ne_18_1_landsat-leaf-on.tif /home/users/jeafilho/dataset/cvpr_chesapeake_landcover/de_1m_2013_extended-debuffered-train_tiles/m_3807503_ne_18_1_lc.tif
(6376, 4865, 9) (6376, 4865)


In [82]:
imgPath = pathlib.Path('/home/users/jeafilho/dataset/cvpr_chesapeake_landcover/de_1m_2013_extended-debuffered-train_tiles/m_3807505_sw_18_1_landsat-leaf-on.tif')
maskPath = pathlib.Path('/home/users/jeafilho/dataset/cvpr_chesapeake_landcover/de_1m_2013_extended-debuffered-train_tiles/m_3807505_sw_18_1_lc.tif')
imgObj = pyvips.Image.new_from_file(str(imgPath))
maskObj = pyvips.Image.new_from_file(str(maskPath))
img = imgObj.numpy()
mask = maskObj.numpy()

In [83]:
print(img.shape, img.min(), img.max())
print(mask.shape, mask.min(), mask.max())

(6364, 4856, 9) -11.5 6964.0
(6364, 4856) 1 6


In [84]:
for i in range(img.shape[2]):
    scaler = MinMaxScaler()
    img[:,:,i] = scaler.fit_transform(np.squeeze(img[:,:,i]))


In [85]:
xlen = 256
ylen = 256
dest = []
img = img.astype(np.float32)
mask = mask.astype(np.float32)
for i in range(img.shape[0]//xlen):
    for j in range(img.shape[1]//ylen):
        _a = np.concatenate([img[xlen*i:xlen*(i+1),ylen*j:ylen*(j+1)],np.expand_dims(mask[xlen*i:xlen*(i+1),ylen*j:ylen*(j+1)], 2)],2)
        dest.append(_a)
final = np.stack(dest)
final.shape

(432, 256, 256, 10)

In [86]:
np.save('/home/users/jeafilho/dataset/original5', final)